In [1]:
import pandas as pd
import numpy as np
import json
import os
import scipy, scipy.io


import matplotlib
from matplotlib import *
from matplotlib import pyplot as plt
import itertools
from mpl_toolkits.axes_grid1 import make_axes_locatable

# pretty charting
# import seaborn as sns
# sns.set_palette('muted')
# sns.set_style('darkgrid')

%matplotlib inline

In [2]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], scipy.io.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        elif isinstance(elem,np.ndarray):
            dict[strg] = _tolist(elem)
        else:
            dict[strg] = elem
    return dict

def _tolist(ndarray):
    '''
    A recursive function which constructs lists from cellarrays 
    (which are loaded as numpy ndarrays), recursing into the elements
    if they contain matobjects.
    '''
    elem_list = []            
    for sub_elem in ndarray:
        if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
            elem_list.append(_todict(sub_elem))
        elif isinstance(sub_elem,np.ndarray):
            elem_list.append(_tolist(sub_elem))
        else:
            elem_list.append(sub_elem)
    return elem_list

def convertMatToJSON(matData, fileName):
    for key in matData.keys():
        if (type(matData[key])) is np.ndarray:
            serializedData = pickle.dumps(matData[key], protocol=0) # protocol 0 is printable ASCII
            jsonData[key] = serializedData
        else:
            jsonData[key] = matData[key]

    with contextlib.closing(bz2.BZ2File(fileName, 'wb')) as f:
        json.dump(jsonData, f)

In [5]:
def degreeofagree(ezset, cezset, allset):
    notcez = list(set(allset) - set(cezset))
    cez_int_eez = list(set(cezset) & set(ezset))
    notcez_int_eez = list(set(notcez) - set(ezset))
    
    term1 = len(cez_int_eez) / len(cezset)
    term2 = len(notcez_int_eez) / len(notcez)
    
    doa = term1 - term2
    return doa

In [4]:
dataDir = '/Users/adam2392/Documents/adamli/fragility_dataanalysis/' + \
    'figures/fragilityStats/notchfilter/perturbationC_win250_step125_radius1.5/interictal'
filelist = os.listdir(dataDir)
print filelist

['pt2aw2_fragilitystats.mat', 'pt2aslp2_fragilitystats.mat', 'pt2aslp1_fragilitystats.mat', 'pt2aw1_fragilitystats.mat', 'pt1aslp1_fragilitystats.mat', 'pt1aslp2_fragilitystats.mat', 'pt3aslp1_fragilitystats.mat', 'pt1aw1_fragilitystats.mat', 'pt3aw1_fragilitystats.mat', 'pt1aw2_fragilitystats.mat', 'pt3aslp2_fragilitystats.mat']


In [17]:
# thresholds to set on the preictal, ictal times of coefficient of variation / other frag stats
thresholds = [0.7, 0.8, 0.9, 0.95, 0.99]

nih_doas = np.zeros((len(filelist), len(thresholds)))

# Load each dataset's fragility statistics results
for idx, fil in enumerate(filelist):
    datastruct = loadmat(os.path.join(dataDir, fil))
    features = datastruct['features_struct']
    
    colnames = features.keys()
    for col in colnames:
        try:
            features[col] = features[col]/max(features[col])
        except:
            print col, " does not work"
    # extract all the fragility stats
    patient = features['patient']
    min_frag = features['min_frag']
    max_frag = features['max_frag']
    ez_asymmetry = features['ez_asymmetry']
    resected_asymmetry = features['resected_asymmetry']
    network_fragility = features['network_fragility']
    cfvar_time = features['cfvar_time']
    cfvar_chan = features['cfvar_chan']
    channels = features['included_labels']
    cezset = features['ezone_labels']
    
    # only if the dataset was ictal
#     precfvar_chan = features['precfvar_chan']
#     postcfvar_chan = features['postcfvar_chan']
#     post20cfvar_chan = features['post20cfvar_chan']
#     post30cfvar_chan = features['post30cfvar_chan']
#     post40cfvar_chan = features['post40cfvar_chan']
#     post50cfvar_chan = features['post50cfvar_chan']
    
    # go through each threshold and determine set of electrodes within EZ set
    print cfvar_chan
    
    # extract the clinical meta data for this patient
    
    pat_doas = np.zeros((1, len(threholds)))
    for jdx, threshold in enumerate(thresholds):
        cfvar_threshed = cfvar_chan[cfvar_chan >= threshold]
        cfvar_chan = channels[cfvar_chan >= threshold]
        
        # compute doa
        doa_threshold = degreeofagree(cfvar_chan, cezset, channels)
        
        # store the channel for this threshold
        pat_doas[0, jdx] = doa_threshold
        break
    break
    
    
    print max(cfvar_chan)
    print max(precfvar_chan)
    print max(postcfvar_chan)
    # create an array of the coefficient of variations of this patient 
    rowarray = np.array(([cfvar_chan, precfvar_chan, \
                         postcfvar_chan, post20cfvar_chan, \
                         post30cfvar_chan, post40cfvar_chan, post50cfvar_chan]))
    print patient
    if idx == 0:
        feature_df = pd.DataFrame(rowarray, columns=['cf var', 'pre cf var', 'post10 cfv', \
                                                    'post20 cfv', 'post30 cfv', \
                                                    'post40 cfv', 'post50 cfv'])
    else:
        row_df = pd.DataFrame(rowarray, ignore_index=True)
        feature_df.append(row_df, ignore_index=True)
    
    display(feature_df)
    break

patient  does not work
min_frag  does not work
max_frag  does not work
ez_asymmetry  does not work
resected_asymmetry  does not work
network_fragility  does not work
[ 0.27357749  0.30325798  0.24111201  0.22957369  0.04896799  0.03830918
  0.30077524  0.31213898  0.21868333  0.30065526  0.14204959  0.28387752
  0.34611963  0.26452052  0.31041356  0.26842786  0.15048332  0.24605565
  0.30406817  0.32804863  0.22440111  0.12967992  0.30125781  0.2022697
  0.02060447  0.17987869  0.22933054  0.28340147  0.30243579  0.2640878
  0.50415134  0.06476604  0.08212339  0.28748599  0.19657617  0.05409319
  0.35445641  0.31553575  0.54335985  0.26957417  0.3367482   0.26456796
  1.          0.36076984  0.36313026  0.22209041  0.28282473  0.39524566
  0.32659685  0.40405091  0.02272254  0.31292851  0.32097206  0.24192412
  0.34111468  0.36505899  0.34099752  0.25347234  0.28330824  0.34822033
  0.3619783   0.37849143  0.31315354]
[ 1.]


In [18]:
print features.keys()

['patient', 'min_frag', 'max_frag', 'ez_asymmetry', 'resected_asymmetry', 'network_fragility', 'cfvar_chan', 'high_frag', 'cfvar_time']
